In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

In [2]:
df = pds.random_data(15_000, n_cols=4)
df.head()

row_num,feature_1,feature_2,feature_3,feature_4
u64,f64,f64,f64,f64
0,0.885231,0.42604,0.868292,0.153928
1,0.785083,0.344915,0.741281,0.602304
2,0.445297,0.532952,0.79264,0.466161
3,0.846072,0.459316,0.412093,0.318678
4,0.39145,0.650961,0.192166,0.754177


In [3]:
df = df.select(
    pl.col("feature_1").alias("B_1"),
    pl.col("feature_2").alias("B_2"),
    pl.col("feature_3").alias("B_3"),
    pl.col("feature_4").alias("A"),
)
df.head()

B_1,B_2,B_3,A
f64,f64,f64,f64
0.885231,0.42604,0.868292,0.153928
0.785083,0.344915,0.741281,0.602304
0.445297,0.532952,0.79264,0.466161
0.846072,0.459316,0.412093,0.318678
0.39145,0.650961,0.192166,0.754177


In [6]:
df.select(
    pds.query_lstsq(pl.col("B_1"), pl.col("B_2"), pl.col("B_3"), target = pl.col("A"), return_pred=True, add_bias=True)
)

A
struct[2]
"{0.507691,-0.353763}"
"{0.505482,0.096821}"
"{0.503406,-0.037246}"
"{0.505341,-0.186664}"
"{0.500792,0.253386}"
…
"{0.497394,-0.276583}"
"{0.505824,0.486579}"
"{0.50807,0.3807}"


In [ ]:
df.with_columns(
    pl.col('A').num.lstsq(*[pl.col(c) for c in df.columns if c.startswith("B_")], return_pred=True).struct.field('resid')
)

In [ ]:
# pl.col('A').num.lstsq(pl.col('^B_.*$'), return_pred=True).struct.field('resid')

In [ ]:
print(df.columns)
df.with_columns(
    pl.col('A').num.lstsq(pl.col('^B_.*$'), return_pred=True) # .struct.field('resid')
).head()

In [ ]:
print(
    
df.select(
    pl.col("row_num"),
    pds.query_radius_ptwise(
        pl.col("feature_1"), pl.col("feature_2"), pl.col("feature_3"), # Columns used as the coordinates in n-d space
        index = pl.col("row_num"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

In [ ]:
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True

In [ ]:
import scipy

In [ ]:
df.select(pl.col("feature_1").num.rfft())

In [ ]:
df.select(pl.col("feature_1").num.rfft2())

In [ ]:
import polars as pl
import polars_ds as pld  # noqa
from pandas._testing import makeTimeDataFrame

In [ ]:
df = makeTimeDataFrame()
df = df.rename(columns={'B': 'B_1', 'C': 'B_2', 'D': 'B_3', })
df = pl.from_pandas(df, include_index=True)